In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np
import gmaps
import requests
import os
# Import the API key.
from config import g_key

In [2]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)
g_baseurl = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chokurdakh,RU,70.6333,147.9167,21.27,87,100,11.30,overcast clouds
1,1,Nikolayevsk-Na-Amure,RU,53.1406,140.7300,32.68,82,61,3.33,broken clouds
2,2,Marzuq,YE,14.4000,46.4667,79.56,51,4,2.64,clear sky
3,3,Viedma,AR,-40.8135,-62.9967,49.19,60,92,19.98,overcast clouds
4,4,Katsuura,JP,35.1333,140.3000,72.19,80,70,8.77,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Marzuq,YE,14.4000,46.4667,79.56,51,4,2.64,clear sky
6,6,Isangel,VU,-19.5500,169.2667,80.71,89,75,0.00,broken clouds
9,9,Hilo,US,19.7297,-155.0900,81.81,74,20,8.05,light rain
14,14,Bubaque,GW,11.2833,-15.8333,83.32,71,99,2.55,overcast clouds
22,22,Bontang,ID,0.1333,117.5000,76.30,92,89,7.70,overcast clouds
29,29,Pacific Grove,US,36.6177,-121.9166,81.81,73,1,5.99,clear sky
32,32,Harper,LR,4.3750,-7.7169,77.58,87,12,7.34,few clouds
33,33,Acarau,BR,-2.8856,-40.1200,79.45,82,49,18.70,scattered clouds
39,39,Campo Maior,BR,-4.8278,-42.1686,90.61,34,100,2.21,overcast clouds
42,42,Rio Claro,BR,-22.4114,-47.5614,82.22,62,15,13.98,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                260
City                   260
Country                259
Lat                    260
Lng                    260
Max Temp               260
Humidity               260
Cloudiness             260
Wind Speed             260
Current Description    260
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))

259


In [8]:
# Show new clean dataframe
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Marzuq,YE,14.4000,46.4667,79.56,51,4,2.64,clear sky
6,6,Isangel,VU,-19.5500,169.2667,80.71,89,75,0.00,broken clouds
9,9,Hilo,US,19.7297,-155.0900,81.81,74,20,8.05,light rain
14,14,Bubaque,GW,11.2833,-15.8333,83.32,71,99,2.55,overcast clouds
22,22,Bontang,ID,0.1333,117.5000,76.30,92,89,7.70,overcast clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
2,Marzuq,YE,79.56,14.4000,46.4667,clear sky,
6,Isangel,VU,80.71,-19.5500,169.2667,broken clouds,
9,Hilo,US,81.81,19.7297,-155.0900,light rain,
14,Bubaque,GW,83.32,11.2833,-15.8333,overcast clouds,
22,Bontang,ID,76.30,0.1333,117.5000,overcast clouds,
29,Pacific Grove,US,81.81,36.6177,-121.9166,clear sky,
32,Harper,LR,77.58,4.3750,-7.7169,few clouds,
33,Acarau,BR,79.45,-2.8856,-40.1200,scattered clouds,
39,Campo Maior,BR,90.61,-4.8278,-42.1686,overcast clouds,
42,Rio Claro,BR,82.22,-22.4114,-47.5614,few clouds,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    g_baseurl = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(g_baseurl, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.loc[index, "Hotel Name"] = np.nan  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
6,Isangel,VU,80.71,-19.5500,169.2667,broken clouds,Tanna Lodge
9,Hilo,US,81.81,19.7297,-155.0900,light rain,Hilo Hawaiian Hotel
14,Bubaque,GW,83.32,11.2833,-15.8333,overcast clouds,"Casa Dora, Bubaque"
22,Bontang,ID,76.30,0.1333,117.5000,overcast clouds,KOPKAR Hotel Bintang Sintuk
29,Pacific Grove,US,81.81,36.6177,-121.9166,clear sky,Lovers Point Inn
32,Harper,LR,77.58,4.3750,-7.7169,few clouds,Screensaver
33,Acarau,BR,79.45,-2.8856,-40.1200,scattered clouds,castelo pizzaria
39,Campo Maior,BR,90.61,-4.8278,-42.1686,overcast clouds,Elite Hotel
42,Rio Claro,BR,82.22,-22.4114,-47.5614,few clouds,Hotel Vivendas Rio Claro by Atlantica
44,Kapaa,US,82.72,22.0752,-159.3190,few clouds,Sheraton Kauai Coconut Beach Resort


In [12]:
len(clean_hotel_df)

239

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))